# Stock Market Analysis Project - Group 12

![](https://cdn-images-1.medium.com/fit/t/1600/480/1*veW_bUCkcsHZ32GMCVhyLA.png)

In [1]:
# Team members:
full_name_1 = "Maggie Wei"
full_name_2 = "Asif Rashid"
full_name_3 = "Candice Chen"


In [46]:
#!pip install yfinance
#!pip install altair

In [4]:
# import all required libraries here
import yfinance as yf
import pandas as pd
from datetime import datetime, date #library datetime helps to return date and time based on specific time zones rather than only in local time
from dateutil.relativedelta import relativedelta
from stocksymbol import StockSymbol
import numpy as np
from scipy.stats import skew
import matplotlib.pyplot as plt
import altair as alt

---
### yfinance library

You can use **yfinance** in your project for downloading data from yahoo finance website. For more information on **yfinance** library go to https://pypi.org/project/yfinance/.

The **download** method returns the prices in Panda dataframe format. The first row of the returned df is the prices of the start date, and the last row is the prices of one day before the end date. The end date is not included in the returned df. For all sections, you can assume the end date refers to one day before the end day (start date up to the end date, end date is not included). See the example below.

---
## Part 1 - Stock Analysis: 

Write a Python statement that can generate a comprehensive report for a given stock. You can use any Python library for this part, but you must use at least **two** new libraries we have not covered in class. The report may generate the following information:
* The current stock price.
* The current market cap.
* Measures such as EPS, PE Ratio, etc.
* At least 3 informative graphs.
* Company Profile.
* Any other relative information such as news about the company.

Be creative and try to collect as much data as you can. Test your code for AAPL, MSFT, and JPM stocks.

**Note:** NumPy is not considered as a new library. You can webscape any website; however, you are not allowed to webscape the websites covered in class.

In [ ]:
# Type the name of the new libraries you use here:
library1 = "scipy.stats"
library2 = "stocksymbol"
library3 = "datetime"
library4 = "dateutil.relativedelta"
library5 = "altair"


In [19]:
def company_details(tickers):
    stock = yf.Ticker(tickers)
    financial_data = stock.info    
    # Display the company profile
    print("Company Name:", financial_data.get("longName"))
    print("Industry:", financial_data.get("industry"))
    print("Sector:", financial_data.get("sector"))
    print("Website:", financial_data.get("website"))
    print("Description:", financial_data.get("longBusinessSummary"))
    print('\n')
    print("Current Stock Price:",financial_data.get("currentPrice", "N/A"))
    print("EPS:",financial_data.get("trailingEps", "N/A"))
    print("Market Cap:",financial_data.get("marketCap", "N/A"))
    print("PE Ratio:",(financial_data.get("currentPrice", None)/financial_data.get("trailingEps", None)))
    print("Price/Book:",financial_data.get("priceToBook", "N/A"))
    print("Return on Equity (ttm):",financial_data.get("returnOnEquity", "N/A"))
    print("Total Debt/Equity (mrq):",financial_data.get("debtToEquity", 0)/100)
    print("Current Ratio (mrq):",financial_data.get("currentRatio", 0))
    print("Total Debt (mrq):",financial_data.get("totalDebt", "N/A"))
    balance_sheet = stock.balance_sheet
    total_assets = balance_sheet.iloc[balance_sheet.index.get_loc("Total Assets"), 0]
    print("Total Asset: ",total_assets)

def generate_combined_charts(ticker, start_time, end_time):
    df = yf.download(ticker, start=start_time, end=end_time) #Get historical stock data
    df.columns = [col if isinstance(col, str) else col[0] for col in df.columns] # Ensure all column names are simple strings

    # Chart 1: Stock Price Over Time: historical trend of the stock's closing price
    stock_price_chart = alt.Chart(df.reset_index()).mark_line().encode(
        x='Date:T',
        y='Close:Q',
        tooltip=['Date:T', 'Close:Q']
    ).properties(
        title=f"{ticker} Stock Price ({start_time} - {end_time})",
        width=1500,
        height=500
    )

    # Chart 2: Daily Percentage Change: day-to-day percentage change in stock prices
    df['Daily Change (%)'] = df['Close'].pct_change() * 100
    daily_change_chart = alt.Chart(df.reset_index()).mark_bar().encode(
        x='Date:T',
        y='Daily Change (%):Q',
        tooltip=['Date:T', 'Daily Change (%):Q']
    ).properties(
        title=f"{ticker} Daily Percentage Change ({start_time} - {end_time})",
        width=1500,
        height=500
    )

    # Chart 3: Trading Volume: volume of stocks traded over time 
    trading_volume_chart = alt.Chart(df.reset_index()).mark_area().encode(
        x='Date:T',
        y='Volume:Q',
        tooltip=['Date:T', 'Volume:Q']
    ).properties(
        title=f"{ticker} Trading Volume ({start_time} - {end_time})",
        width=1500,
        height=500
    )

    # Combine the charts vertically
    combined_chart = alt.vconcat(
        stock_price_chart,
        daily_change_chart,
        trading_volume_chart
    ).properties(
        title=f"{ticker} Combined Charts ({start_time} - {end_time})"
    )

    # Save combined chart to an HTML file
    combined_chart.save(f"{ticker}_combined_charts.html")
    #print(f"Combined charts saved as {ticker}_combined_charts.html")
    return combined_chart






In [20]:
tickers = "AAPL"
company_details(tickers)
generate_combined_charts(tickers, "2020-01-01", "2024-01-01")

Company Name: Apple Inc.
Industry: Consumer Electronics
Sector: Technology
Website: https://www.apple.com
Description: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; App

[*********************100%***********************]  1 of 1 completed


alt.VConcatChart(...)

---
## Part 2 - Automated Investment Platform

Develop a Python program that takes a list of tickers and analyzes them. The automated investment platform should pick one stock to **BUY**. 

* Design an investment strategy and briefly explain that.
* The program should print out a report comparing the stocks.
* The program should pick the stock based on a logical and well-justified strategy. So you cannot randomly select a stock.

You can use any Python library for this part. First, develop an investment strategy and then convert it into a program. You should clearly explain your investment strategy and the required steps for implementation.

In [21]:
# Your program goes here
def get_financial_data(tickers):
    all_data = []
    
    for ticker in tickers:
        unretrievable = 0
        
        try:

            stock = yf.Ticker(ticker)
            financial_data = stock.info

            current_price = financial_data.get("currentPrice", None)
            trailing_eps = financial_data.get("trailingEps", None)
            pe_ratio = current_price / trailing_eps

            balance_sheet = stock.balance_sheet
            total_assets = balance_sheet.iloc[balance_sheet.index.get_loc("Total Assets"), 0]
            data = {
                "Ticker": ticker,
                "Current Stock Price": '${:,.2f}'.format(current_price),
                "Current Market Cap (in billions)": '${:,}'.format(round(financial_data.get("marketCap", "N/A")/1000000000)),
                "Total Assets (in billions)": '${:,}'.format(round(total_assets/1000000000)),                
                "Total Debt (in billions) (mrq)": '${:,}'.format(round(financial_data.get("totalDebt", "N/A")/1000000000)),
                "Current EPS": trailing_eps,
                "P/E Ratio": round(pe_ratio,3),
                "Price/Book": round(financial_data.get("priceToBook", "N/A"),3),
                "Return on Equity (ttm)": round(financial_data.get("returnOnEquity", "N/A"),3),
                "Total Debt/Equity (mrq)": round(financial_data.get("debtToEquity", "N/A")/100,3), #percent to decimal
                "Debt to Asset": round(financial_data.get("totalDebt", "N/A") / total_assets,4),
                "Current Ratio (mrq)": financial_data.get("currentRatio", "N/A"),
                "EPS YoY Growth": round(((trailing_eps/financial_data.get("forwardEps", "N/A"))- 1),3)
            }

            all_data.append(data)
            

        except Exception as e:
            unretrievable += 1 
            if unretrievable == 1:
                print(f"Required metrics can not be retrieved for {ticker} ")
            elif unretrievable > 1:
                continue

    df = pd.DataFrame(all_data)
    return df

In [22]:
def get_tickers(n):
    api_key = '3aa2f8d3-ecdc-407e-a273-0a7d9cca08f5 '
    ss = StockSymbol(api_key)
    
    # get symbol list based on market
    symbol_list_us = ss.get_symbol_list(market="US") # "us" or "america" will also work
    
    x=n
    tickers_db=pd.DataFrame(symbol_list_us)
    ticks=tickers_db['symbol']
    ticks[6]='META'
    ticks_updated=[]
    for ticker_rename in ticks:
        ticks_updated.append(ticker_rename.replace('.','-'))
    ticks_test=ticks_updated[:x]
    
    ticker_name=tickers_db['longName'].head(x)
    company_name_list=[]
    for i in ticker_name:
        company_name_list.append(i)

    return ticks_test

In [23]:
# Your program goes here
def Buy_strategy(tickers): 
    
    #Part 1: Medium Momentum Strategy
    today = date.today()
    start_time = today - relativedelta(years=1) #start time is 1 year ago from today
    end_time = today - relativedelta(months=2) #end time is 2 months ago from today
    removed_part1 = []
    
    momentum = []
    
    for ticker in tickers:
        try:
            Adjusted_Closing = yf.download(ticker, start=str(start_time), end=str(end_time), progress= False)["Adj Close"]
            time_index = np.arange(len(Adjusted_Closing))
            slope = np.polyfit(time_index, Adjusted_Closing, 1)[0][0]
            
            if slope < 0:  #if slope is negative, add to list of tickers to remove
                removed_part1.append(ticker)
                
        except Exception as e:
            print(f"Required prices for slope can not be retrieved for {ticker} ")
            continue
    
    
    if len(removed_part1) != 0:
        print(f"********************************************************************"\
              f"\nThe following stocks did not pass the positive momentum screening: {', '.join(removed_part1)}"\
              f"\n********************************************************************")
    
    for ticker in removed_part1:
        tickers.remove(ticker)

    #Part 2: Prospect Theory
    removed_part2 = []
    storage = []
    
    for ticker in tickers: 
        Points = 0
        stock = yf.Ticker(ticker)
        
        #beta 
        beta = stock.info.get('beta', "N/A")    
        try: 
            if beta > 1:
                Points += 1 
        except Exception as e:
            print(f"Required beta can not be retrieved for {ticker} ")
            continue
        
        #skewness
        start_time = today - relativedelta(years=1) #start time is 1 year ago from today
        prices = yf.download(ticker, start=str(start_time), end=str(today), progress = False) #end time is today
        prices["Daily Returns"] = prices["Adj Close"].pct_change() #Daily returns is % change of adjusted closing price
        skewness = skew(prices["Daily Returns"], nan_policy= "omit")
        if skewness < 0:
            Points += 1 
        
        #capital gain overhang
        current_price = yf.Ticker(ticker).info.get("currentPrice", None)
        one_year_price = prices.head(1)["Adj Close"].values[0][0]
        capital_gain = (current_price/one_year_price)-1
        if capital_gain > 0:
            Points += 1 
    
        #Storing data in dataframe for comparison
        data = {"Ticker": ticker, 
                "Beta": beta, 
                "Skewness": skewness,
                "Capital Gain": capital_gain,
                "Points": Points}
    
        storage.append(data)
    
        #Storing data in dataframe for comparison
        if Points < 2:
            removed_part2.append(ticker)
    
    part2 = pd.DataFrame(storage).reset_index(drop=True).sort_values("Points").head(10)
    
    print(f"********************************************************************"\
          f"\nSample of filter based on beta, skewness of daily returns and capital gain overhang"\
          f"\n********************************************************************")
        
    display(part2)
    
    if len(removed_part2) != 0:
        print(f"********************************************************************"\
              f"\nThe following stocks did not pass the prospect theory filter: {', '.join(removed_part2)}"\
              f"\n********************************************************************")
    
    for ticker in removed_part2:
        tickers.remove(ticker)
    
    #Part 3: Fundamental Analysis
    df = get_financial_data(tickers)
    Points = [0] * len(df.Ticker)
    
    for i in range(len(df.Ticker)):
        #Price_to_Equity
        PE = df.loc[i, "P/E Ratio"]
        if PE  >= 20:
            Points[i] += 0
        elif 10 < PE < 20:
            Points[i] += 0.1
        elif PE <= 10:
            Points[i] += 1
        
        #Price_to_Book
        PB = df.loc[i, "Price/Book"]
        if PB >= 3:
            Points[i] += 0
        elif 1 < PB < 3:
            Points[i] += 0.5
        elif PB <= 1:
            Points[i] += 1
        
        #EPS YoY Growth
        EPS_Growth = df.loc[i, "EPS YoY Growth"]
        if EPS_Growth >= 0.1:
            Points[i] += 1
        elif 0 < EPS_Growth < 0.1:
            Points[i] += 0.5
        elif EPS_Growth < 0:
            Points[i] += 0
        
        #Return on Equity (ttm)
        ROE = df.loc[i, "Return on Equity (ttm)"]
        if ROE >= 0.15:
            Points[i] += 1
        elif 0.1 < ROE < 0.15:
            Points[i] += 0.5
        elif ROE < 0:
            Points[i] += 0
        
        #Current Ratio
        Current = df.loc[i, "Current Ratio (mrq)"]
        if Current >= 2:
            Points[i] += 1
        elif 1 < Current < 2:
            Points[i] += 0.5
        elif Current < 1:
            Points[i] += 0
        
        #Debt-to-Equity
        DE = df.loc[i, "Total Debt/Equity (mrq)"]
        if DE >= 2:
            Points[i] += 0
        elif 1 < DE < 2:
            Points[i] += 0.5
        elif DE < 1:
            Points[i] += 1
        
        #Debt-to-Asset
        DA = df.loc[i, "Debt to Asset"]
        if DA >= 0.5:
            Points[i] += 0
        elif 0.25 < DA < 0.5:
            Points[i] += 0.5
        elif DA < 0.25:
            Points[i] += 1
    
    df["Points"] = Points
    
    Points_sorted = df.sort_values("Points", ascending = False)
    
    print(f"********************************************************************"\
          f"\nFundamental analysis table on stocks that passed the behavioral screening"\
          f"\n(Top 20 scores assigned based on health, performance, and valuation):"\
          f"\n********************************************************************")
    
    display(Points_sorted.head(20))

    #Part 4: Final Selection based on ROE if necessary 
    max_points = Points_sorted.max().Points
    max_ROE = 0
    Buy = []
    Final = ""
    tickers = list(Points_sorted.Ticker)
    
    for ticker in tickers:
        point = Points_sorted[(Points_sorted.Ticker == ticker)]["Points"].values[0]
        if point == max_points:
            Buy.append(ticker)
    
    if len(Buy) == 1:
        return print(f"********************************************************************"\
          f"\nWith the highest fundamental analysis score of {max_points}, the recommended stock to buy is: {', '.join(Buy)}"\
          f"\n********************************************************************")
    elif len(Buy) > 1:
        print(f"********************************************************************"\
              f"\n{len(Buy)} stocks are tied with the highest fundamental analysis scores of {max_points}: {', '.join(Buy)}")
        for ticker in Buy:
            ROE = Points_sorted[(Points_sorted.Ticker == ticker)]["Return on Equity (ttm)"].values[0]
            if ROE >= max_ROE:
                max_ROE = ROE
                Final = ticker
        return print(f"********************************************************************"\
          f"\nWith the higher ROE, the recommended stock to buy is: {Final}"\
          f"\n********************************************************************")


In [24]:
Buy_strategy(get_tickers(30))

********************************************************************
The following stocks did not pass the positive momentum screening: DIS, TM
********************************************************************
********************************************************************
Sample of filter based on beta, skewness of daily returns and capital gain overhang
********************************************************************


,Ticker,Beta,Skewness,Capital Gain,Points
24,PFE,0.615,0.125490,-0.064355,0
12,JNJ,0.518,0.533216,-0.027640,0
23,RHHBF,0.145,0.206397,0.073832,1
22,RHHVF,0.145,0.517082,0.056373,1
21,BABA,0.349,0.516830,0.205279,1
20,RHHBY,0.145,0.097580,0.056969,1
7,BRK-B,0.871,0.243228,0.325156,1
8,BRK-A,0.871,0.233905,0.308352,1
17,WMT,0.516,1.643838,0.864659,1
0,AAPL,1.240,0.490277,0.260280,2


********************************************************************
The following stocks did not pass the prospect theory filter: BRK-B, BRK-A, JNJ, WMT, RHHBY, BABA, RHHVF, RHHBF, PFE
********************************************************************
Required metrics can not be retrieved for JPM 
Required metrics can not be retrieved for BAC 
********************************************************************
Fundamental analysis table on stocks that passed the behavioral screening
(Top 20 scores assigned based on health, performance, and valuation):
********************************************************************


,Ticker,Current Stock Price,Current Market Cap (in billions),Total Assets (in billions),Total Debt (in billions) (mrq),Current EPS,P/E Ratio,Price/Book,Return on Equity (ttm),Total Debt/Equity (mrq),Debt to Asset,Current Ratio (mrq),EPS YoY Growth,Points
8,TSM,$204.87,"$1,063","$5,532",$969,6.22,32.937,1.331,0.280,0.241,0.1751,2.567,-0.230,4.5
5,TSLA,$369.50,"$1,186",$107,$13,3.65,101.233,16.945,0.204,0.181,0.1199,1.844,0.127,4.5
15,XOM,$114.56,$504,$376,$43,8.03,14.267,1.875,0.145,0.154,0.1131,1.348,0.020,4.1
6,META,$611.50,"$1,544",$230,$49,21.18,28.872,9.381,0.361,0.298,0.2136,2.732,-0.163,4.0
7,NVDA,$146.06,"$3,577",$66,$10,2.54,57.502,83.747,1.238,0.172,0.1524,4.269,-0.383,4.0
2,GOOG,$175.34,"$2,135",$402,$29,7.54,23.255,6.846,0.321,0.093,0.0728,1.950,-0.158,3.5
3,GOOGL,$173.65,"$2,135",$402,$29,7.53,23.061,6.780,0.321,0.093,0.0728,1.950,-0.160,3.5
1,MSFT,$441.31,"$3,281",$512,$97,12.12,36.412,11.405,0.356,0.337,0.1891,1.301,-0.189,3.5
9,V,$310.50,$601,$95,$21,9.73,31.912,15.465,0.507,0.532,0.2205,1.283,-0.231,3.5
14,ASML,$715.21,$281,$40,$5,18.50,38.660,17.411,0.492,0.291,0.1174,1.551,-0.307,3.5


********************************************************************
2 stocks are tied with the highest fundamental analysis scores of 4.5: TSM, TSLA
********************************************************************
With the higher ROE, the recommended stock to buy is: TSM
********************************************************************


---
## Part 3 - Automated Investment Platform

Develop a Python program that takes a portfolio (list of tickers) and analyzes them. The automated investment platform should pick one stock to **SELL**. 

* Design an investment strategy and briefly explain that.
* The program should print out a report comparing the stocks.
* The program should pick the stock based on a logical and well-justified strategy. So you cannot randomly select a stock.

You can use any Python library for this part. First, develop an investment strategy and then convert it into a program. You should clearly explain your investment strategy and the required steps for implementation.

In [26]:
# Your program goes here
def Sell_strategy(tickers):

    #Part 1: Financial Analysis
    df = get_financial_data(tickers)
    min_points = 50
    part1_sell = []
    Points = [0] * len(df.Ticker)
    
    for i in range(len(df.Ticker)):
        #Price_to_Equity
        PE = df.loc[i, "P/E Ratio"]
        if PE  >= 20:
            Points[i] += 0
        elif 10 < PE < 20:
            Points[i] += 0.1
        elif PE <= 10:
            Points[i] += 1
        
        #Price_to_Book
        PB = df.loc[i, "Price/Book"]
        if PB >= 3:
            Points[i] += 0
        elif 1 < PB < 3:
            Points[i] += 0.5
        elif PB <= 1:
            Points[i] += 1
        
        #EPS YoY Growth
        EPS_Growth = df.loc[i, "EPS YoY Growth"]
        if EPS_Growth >= 0.1:
            Points[i] += 1
        elif 0 < EPS_Growth < 0.1:
            Points[i] += 0.5
        elif EPS_Growth < 0:
            Points[i] += 0
        
        #Return on Equity (ttm)
        ROE = df.loc[i, "Return on Equity (ttm)"]
        if ROE >= 0.15:
            Points[i] += 1
        elif 0.1 < ROE < 0.15:
            Points[i] += 0.5
        elif ROE < 0:
            Points[i] += 0
        
        #Current Ratio
        Current = df.loc[i, "Current Ratio (mrq)"]
        if Current >= 2:
            Points[i] += 1
        elif 1 < Current < 2:
            Points[i] += 0.5
        elif Current < 1:
            Points[i] += 0
        
        #Debt-to-Equity
        DE = df.loc[i, "Total Debt/Equity (mrq)"]
        if DE >= 2:
            Points[i] += 0
        elif 1 < DE < 2:
            Points[i] += 0.5
        elif DE < 1:
            Points[i] += 1
        
        #Debt-to-Asset
        DA = df.loc[i, "Debt to Asset"]
        if DA >= 0.5:
            Points[i] += 0
        elif 0.25 < DA < 0.5:
            Points[i] += 0.5
        elif DA < 0.25:
            Points[i] += 1
    
        #Determine min points
        if Points[i] <= min_points:
            min_points = Points[i]
    
    df["Points"] = Points
    Points_sorted = df.sort_values("Points", ascending = True)
    
    #Append stock with minimum points to list
    for i in range(len(df.Ticker)):
        if df.Points[i] == min_points:
            part1_sell.append(df.Ticker[i])
    
    #Display Results of Part 1
    print(f"********************************************************************"\
          f"\nFundamental analysis table on stocks from provided list"\
          f"\n(20 lowest scores assigned based on health, performance, and valuation):"\
          f"\n********************************************************************")

    display(Points_sorted.head(20))

    if len(part1_sell) == 1:
        print(f"********************************************************************"\
              f"\nWith the lowest fundamental analysis score, the recommended stock to sell is: {', '.join(part1_sell)}"\
              f"\n********************************************************************")
    elif len(part1_sell) > 1:
        print(f"********************************************************************"\
              f"\nThe following(s) stock has the lowest fundamental analysis scores: {', '.join(part1_sell)}."\
             f"\nWe will apply behavioral screening to select the recommended stock to sell."\
             f"\n********************************************************************")

    #Part 2: Medium Term Momentum
    today = date.today()
    start_time = today - relativedelta(years=1) #start time is 1 year ago from today
    end_time = today - relativedelta(months=2) #end time is 2 months ago from today
    part2_sell = []
    part2_remove = []
    
    for ticker in part1_sell:
        try: 
            Adjusted_Closing = yf.download(ticker, start=str(start_time), end=str(end_time), progress= False)["Adj Close"]
            time_index = np.arange(len(Adjusted_Closing))
    
            slope = np.polyfit(time_index, Adjusted_Closing, 1)[0][0]
            print(f"Slope of {ticker} momentum: {slope}")
            
            if slope < 0:  
                part2_sell.append(ticker)
            elif slope >0:
                part2_remove.append(ticker)
                
        except Exception as e:
            print(f"Required prices for slope can not be retrieved for {ticker} ")
            continue
    
    if len(part2_sell) == 1:
        print(f"********************************************************************"\
              f"\nWith a negative momentum, the recommended stock to sell is: {', '.join(part2_sell)}."\
              f"\n********************************************************************")
    if len(part2_sell) > 1:
        print(f"********************************************************************"\
              f"\nThe following stocks all have negative momentum: {', '.join(part2_sell)}."\
             f"\nWe will apply prospect theory to select the recommended stock to sell."\
             f"\n********************************************************************")
    if len(part2_sell) == 0:
        print(f"********************************************************************"\
              f"\nThe following stocks all have positive momentum: {', '.join(part2_remove)}."\
             f"\nWe will apply prospect theory to select the recommended stock to sell."\
             f"\n********************************************************************")
        
    #Part 3: Prospect Theory
    part3_remove = []
    storage = []
    part3_sell = []
    
    if len(part2_sell) > 1:
        part2 = part2_sell
    elif len(part2_remove) > 1:
        part2 = part2_remove
    
    for ticker in part2: 
        Points = 0
        stock = yf.Ticker(ticker)
        
        #beta 
        beta = stock.info.get('beta', "N/A")    
        try: 
            if beta > 1:
                Points += 1 
        except Exception as e:
            print(f"Required beta can not be retrieved for {ticker} ")
            continue
        
        #skewness
        start_time = today - relativedelta(years=1) #start time is 1 year ago from today
        prices = yf.download(ticker, start=str(start_time), end=str(today), progress = False) #end time is today
        prices["Daily Returns"] = prices["Adj Close"].pct_change() #Daily returns is % change of adjusted closing price
        skewness = skew(prices["Daily Returns"], nan_policy= "omit")
        if skewness < 0:
            Points += 1 
        
        #capital gain overhang
        current_price = yf.Ticker(ticker).info.get("currentPrice", None)
        one_year_price = prices.head(1)["Adj Close"].values[0][0]
        capital_gain = (current_price/one_year_price)-1
        if capital_gain > 0:
            Points += 1 
    
        #Storing data in dataframe for comparison
        data = {"Ticker": ticker, 
                "Beta": beta, 
                "Skewness": skewness,
                "Capital Gain": capital_gain,
                "Points": Points}
    
        storage.append(data)
        
        #Storing data in dataframe for comparison
        if Points < 2:
            part3_sell.append(ticker)
        elif Points > 2:
            part3_remove.append(ticker)
    
    prospect = pd.DataFrame(storage).reset_index(drop=True).sort_values("Points", ascending = True)
    display(prospect)

    #Part 4: Final Selection based on lowest capital gain
    min_points = prospect["Points"].min()
    min_cg = 50
    tickers = list(prospect.Ticker)
    Sell = []
    Final = ""

    #Append tickers with the lowest scores to list
    for ticker in tickers:
        point = prospect[(prospect.Ticker == ticker)]["Points"].values[0]
        if point == min_points:
            Sell.append(ticker)

    #Choose ticker with lowest capital gain if there are multiple tickers with the same lowest score
    if len(Sell) == 1:
        return print(f"********************************************************************"\
          f"\nWith the lowest prospect theory score, the recommended stock to sell is: {', '.join(Sell)}"\
          f"\n********************************************************************")
    elif len(Sell) > 1:
        print(f"********************************************************************"\
              f"\n{len(Sell)} stocks are tied with the lowest prospect theory scores of {min_points}: {', '.join(Sell)}")
        for ticker in Sell:
            capital_gain = prospect[(prospect.Ticker == ticker)]["Capital Gain"].values[0]
            if capital_gain <= min_cg:
                min_cg = capital_gain
                Final = ticker

        return print(f"********************************************************************"\
          f"\nWith the lowest capital gain, the recommended stock to sell is: {Final}"\
          f"\n********************************************************************")


In [27]:
Sell_strategy(get_tickers(50))

Required metrics can not be retrieved for JPM 
Required metrics can not be retrieved for BAC 
Required metrics can not be retrieved for WFC 
********************************************************************
Fundamental analysis table on stocks from provided list
(20 lowest scores assigned based on health, performance, and valuation):
********************************************************************


,Ticker,Current Stock Price,Current Market Cap (in billions),Total Assets (in billions),Total Debt (in billions) (mrq),Current EPS,P/E Ratio,Price/Book,Return on Equity (ttm),Total Debt/Equity (mrq),Debt to Asset,Current Ratio (mrq),EPS YoY Growth,Points
39,ABBV,$175.66,$310,$135,$71,2.88,60.993,51.468,0.564,11.748,0.5295,0.645,-0.763,1.0
41,ORCL,$187.31,$519,$141,$85,3.88,48.276,47.991,1.556,7.500,0.5995,0.720,-0.458,1.0
0,AAPL,$242.59,"$3,667",$365,$119,6.08,39.900,64.399,1.574,2.091,0.3262,0.867,-0.268,1.5
35,PEP,$159.78,$219,$100,$45,6.78,23.567,11.278,0.488,2.296,0.4479,0.886,-0.214,1.5
14,HD,$427.76,$425,$77,$63,14.72,29.060,73.260,4.049,10.954,0.8281,1.133,-0.056,1.5
29,AVGO,$171.35,$800,$73,$40,1.25,137.080,2.893,0.125,1.660,0.5553,1.038,-0.797,2.0
30,LLY,$827.58,$744,$64,$31,9.29,89.083,52.296,0.653,2.181,0.4879,1.273,-0.590,2.0
22,PFE,$25.63,$145,$227,$68,0.75,34.173,1.574,0.045,0.734,0.3000,1.000,-0.744,2.0
17,MA,$526.39,$483,$42,$18,13.24,39.758,65.092,1.776,2.448,0.4325,1.289,-0.192,2.0
45,VZ,$42.75,$180,$380,$178,2.31,18.506,1.868,0.104,1.826,0.4690,0.657,-0.512,2.1


********************************************************************
The following(s) stock has the lowest fundamental analysis scores: ABBV, ORCL.
We will apply behavioral screening to select the recommended stock to sell.
********************************************************************
Slope of ABBV momentum: 0.1816627683232744
Slope of ORCL momentum: 0.25696062635817835
********************************************************************
The following stocks all have positive momentum: ABBV, ORCL.
We will apply prospect theory to select the recommended stock to sell.
********************************************************************


,Ticker,Beta,Skewness,Capital Gain,Points
0,ABBV,0.613,-2.403221,0.259719,2
1,ORCL,1.013,1.188905,0.655467,2


********************************************************************
2 stocks are tied with the lowest prospect theory scores of 2: ABBV, ORCL
********************************************************************
With the lowest capital gain, the recommended stock to sell is: ABBV
********************************************************************
